In [48]:
import pandas as pd

In [49]:
df = pd.read_csv("/Users/db/Desktop/dishanb/Wharton-HS-Competition-Soccer-Playoff-Predictions/data/given_data/NSL_regular_season_data_2.csv")

dff = pd.concat([df, df], ignore_index=True)
dff.sort_values(by=dff.columns.tolist(), inplace=True)

dff.reset_index(drop=True, inplace=True)

values = [1, 2] * (len(dff) // 2) + [1] * (len(dff) % 2)
dff['number'] = values

fd = pd.DataFrame(columns=['Team', 'goalsFor', 'goalsAgainst', 'xGFor', 'xGAgainst',
                           'shotsFor', 'shotsAgainst', 'cornersFor', 'cornersAgainst',
                           'pkFor', 'pkAgainst', 'pkShotsFor', 'pkShotsAgainst', 'possessionPct', 'isHome', 'Result', 'PointsGained'])

for index, row in dff.iterrows():
    is_home = row['number'] == 1
    fd.at[index, 'Team'] = row['HomeTeam'] if is_home else row['AwayTeam']
    fd.at[index, 'goalsFor'], fd.at[index, 'goalsAgainst'] = (row['HomeScore'], row['AwayScore']) if is_home else (row['AwayScore'], row['HomeScore'])
    fd.at[index, 'xGFor'], fd.at[index, 'xGAgainst'] = (row['Home_xG'], row['Away_xG']) if is_home else (row['Away_xG'], row['Home_xG'])
    fd.at[index, 'shotsFor'], fd.at[index, 'shotsAgainst'] = (row['Home_shots'], row['Away_shots']) if is_home else (row['Away_shots'], row['Home_shots'])
    fd.at[index, 'cornersFor'], fd.at[index, 'cornersAgainst'] = (row['Home_corner'], row['Away_corner']) if is_home else (row['Away_corner'], row['Home_corner'])
    fd.at[index, 'pkFor'], fd.at[index, 'pkAgainst'] = (row['Home_PK_Goal'], row['Away_PK_Goal']) if is_home else (row['Away_PK_Goal'], row['Home_PK_Goal'])
    fd.at[index, 'pkShotsFor'], fd.at[index, 'pkShotsAgainst'] = (row['Home_PK_shots'], row['Away_PK_shots']) if is_home else (row['Away_PK_shots'], row['Home_PK_shots'])
    fd.at[index, 'possessionPct'] = row['Home_ToP'] if is_home else 1 - row['Home_ToP']
    fd.at[index, 'isHome'] = int(is_home)


In [50]:
# Defining a win, loss, or draw
for index, row in fd.iterrows():
    if row['goalsFor'] > row['goalsAgainst']:
        row['Result'] = 1
        row['PointsGained'] = 3
    elif row['goalsAgainst'] > row['goalsFor']:
        row['Result'] = -1
        row['PointsGained'] = 0
    elif row['goalsFor'] == row['goalsAgainst']:
        row['Result'] = 0
        row['PointsGained'] = 1

In [51]:
fd.to_csv("/Users/db/Desktop/dishanb/Wharton-HS-Competition-Soccer-Playoff-Predictions/data/manipulated_data/detailedRegSeasonData.csv")

In [52]:
atHome = fd[fd['isHome'] == 1]
atAway = fd[fd['isHome'] == 0]

unique_teams = fd['Team'].unique()


In [53]:
points_sum_per_team = fd.groupby('Team')['PointsGained'].sum().reset_index()
winCount = fd.groupby('Team')['Result'].apply(lambda x: (x == 1).sum()).reset_index()
lossCount = fd.groupby('Team')['Result'].apply(lambda x: (x == -1).sum()).reset_index()
drawCount = fd.groupby('Team')['Result'].apply(lambda x: (x == 0).sum()).reset_index()

In [54]:
rankings = pd.DataFrame()

In [60]:
rankings['Team'] = points_sum_per_team['Team']
rankings['Points'] = points_sum_per_team['PointsGained']
rankings['Wins'] = winCount['Result']
rankings['Losses'] = lossCount['Result']
rankings['Draws'] = drawCount['Result']

rankings['WinPercentage'] = rankings['Wins'] / (rankings['Wins'] + rankings['Losses'] + rankings['Draws'])
rankings['LossPercentage'] = rankings['Losses'] / (rankings['Wins'] + rankings['Losses'] + rankings['Draws'])
rankings['DrawPercentage'] = rankings['Draws'] / (rankings['Wins'] + rankings['Losses'] + rankings['Draws'])

rankings['GoalDifference'] = fd.groupby('Team')['goalsFor'].sum().reset_index()['goalsFor'] - fd.groupby('Team')['goalsAgainst'].sum().reset_index()['goalsAgainst']
rankings['xGDifference'] = fd.groupby('Team')['xGFor'].sum().reset_index()['xGFor'] - fd.groupby('Team')['xGAgainst'].sum().reset_index()['xGAgainst']
rankings['ShotsFor'] = fd.groupby('Team')['shotsFor'].sum().reset_index()['shotsFor']
rankings['ShotsAgainst'] = fd.groupby('Team')['shotsAgainst'].sum().reset_index()['shotsAgainst']
rankings['CornersFor'] = fd.groupby('Team')['cornersFor'].sum().reset_index()['cornersFor']
rankings['CornersAgainst'] = fd.groupby('Team')['cornersAgainst'].sum().reset_index()['cornersAgainst']
rankings['PKFor'] = fd.groupby('Team')['pkFor'].sum().reset_index()['pkFor']
rankings['PKAgainst'] = fd.groupby('Team')['pkAgainst'].sum().reset_index()['pkAgainst']
rankings['PKShotsFor'] = fd.groupby('Team')['pkShotsFor'].sum().reset_index()['pkShotsFor']
rankings['PKShotsAgainst'] = fd.groupby('Team')['pkShotsAgainst'].sum().reset_index()['pkShotsAgainst']
rankings['PossessionPct'] = fd.groupby('Team')['possessionPct'].mean().reset_index()['possessionPct']



In [61]:
rankings = rankings.sort_values(by='Points', ascending=False)
rankings
rankings.to_csv("/Users/db/Desktop/dishanb/Wharton-HS-Competition-Soccer-Playoff-Predictions/data/manipulated_data/rankings.csv")

In [57]:
overallAvgData = []
for team in unique_teams:
    team_data = fd.loc[fd['Team'] == team]
    team_data_numeric = team_data.apply(pd.to_numeric, errors='coerce')
    team_data_numeric_mean = team_data_numeric.mean().to_dict()
    team_data_numeric_mean['Team'] = team
    overallAvgData.append(team_data_numeric_mean)

overallAverages = pd.DataFrame(overallAvgData)


In [58]:
homeAvgData = []
for team in unique_teams:
    team_data = atHome.loc[atHome['Team'] == team]
    team_data_numeric = team_data.apply(pd.to_numeric, errors='coerce')
    team_data_numeric_mean = team_data_numeric.mean().to_dict()
    team_data_numeric_mean['Team'] = team
    homeAvgData.append(team_data_numeric_mean)
homeAverages = pd.DataFrame(homeAvgData)

awayAvgData = []
for team in unique_teams:
    team_data = atAway.loc[atAway['Team'] == team]
    team_data_numeric = team_data.apply(pd.to_numeric, errors='coerce')
    team_data_numeric_mean = team_data_numeric.mean().to_dict()
    team_data_numeric_mean['Team'] = team
    awayAvgData.append(team_data_numeric_mean)
awayAverages = pd.DataFrame(awayAvgData)

In [59]:
overallAverages.to_csv("/Users/db/Desktop/dishanb/Wharton-HS-Competition-Soccer-Playoff-Predictions/data/manipulated_data/overallAverages.csv")
homeAverages.to_csv("/Users/db/Desktop/dishanb/Wharton-HS-Competition-Soccer-Playoff-Predictions/data/manipulated_data/homeAverages.csv")
awayAverages.to_csv("/Users/db/Desktop/dishanb/Wharton-HS-Competition-Soccer-Playoff-Predictions/data/manipulated_data/awayAverages.csv")